In [1]:
import os
import inro.modeller as m
from copy import deepcopy

In [16]:
mm = m.Modeller()


NAMESPACE = "tmg.input_output.import_binary_matrix"
import_matrices = mm.tool(NAMESPACE)           
NAMESPACE = "inro.emme.data.matrix.change_matrix_properties"
change_matrix = m.Modeller().tool(NAMESPACE)
NAMESPACE = "inro.emme.matrix_calculation.matrix_calculator"
compute_matrix = mm.tool(NAMESPACE)
NAMESPACE = "inro.emme.data.matrix.create_matrix"
create_matrix = mm.tool(NAMESPACE)
NAMESPACE = "inro.emme.data.matrix.delete_matrix"
delete_matrix = mm.tool(NAMESPACE)
NAMESPACE = "inro.emme.data.matrix.export_matrix_to_csv"
export_matrix = mm.tool(NAMESPACE)
eb = m.Modeller().emmebank

inputFolder = r"C:\Users\BCRANE\Documents\GIS"

scen = mm.scenario
year = "2041C"

finalDest = "md53"

calcSpec = {
    "expression": None,
    "result": None,
    "constraint": {
        "by_value": None,
        "by_zone": {
            "destinations": None
        }
    },
    "aggregation": {
        "origins": "+",
        "destinations": None
    },
    "type": "MATRIX_CALCULATION"
}

In [10]:
pAutoFile = os.path.join(inputFolder,"skim.am_peak.auto.sov.time.mdf")
pAutoTime = eb.available_matrix_identifier('FULL')
import_matrices(4, autoTime[2:] ,pAutoFile,mm.scenario, "peakSovtimeCommitted")

opAutoFile = os.path.join(inputFolder,"skim.midday.auto.sov.time.mdf")
opAutoTime = eb.available_matrix_identifier('FULL')
import_matrices(4, autoTime[2:] ,opAutoFile,mm.scenario, "offpeakSovtimeCommitted")


'\nautodemFile = os.path.join(inputFolder,"trips_auto_peak.mdf")\nautoDem = eb.available_matrix_identifier(\'FULL\')\nimport_matrices(4, autoDem[2:] ,autodemFile,mm.scenario, "newtrips_auto_peak" + year)\nchange_matrix(matrix = autoDem,\n              matrix_name = "newtrips_auto_peak" + year,\n              matrix_description = "Trips Auto Peak" + year)\n\ntransitdemFile = os.path.join(inputFolder,"trips_transit_peak.mdf")\ntransitDem = eb.available_matrix_identifier(\'FULL\')\nimport_matrices(4, transitDem[2:] ,transitdemFile,mm.scenario, "newtrips_transit_peak" + year)\nchange_matrix(matrix = transitDem,\n              matrix_name = "newtrips_transit_peak" + year,\n              matrix_description = "Trips Transit Peak" + year)\n\ndemand = eb.available_matrix_identifier(\'FULL\')\ncreate_matrix(matrix_id = demand,\n              matrix_name = "new_at_tripsatpeak_" + year,\n              matrix_description = "P & A Trips Trans Auto Peak " + year )\ndemCalc = deepcopy(calcSpec)\ndemCa

In [ ]:
d1 = "md61"
create_matrix(matrix_id = d1,
              matrix_name = "trucktimeph",
              matrix_description = "Truck Time Placeholder")

In [ ]:
destinations = ["6187","4276","1477","2028"]
for dest in destinations:
    calc1 = deepcopy(calcSpec)
    calc1["expression"] = pAutoTime
    calc1["result"] = d1
    calc1["aggregation"]["origins"] = "+"
    calc1["constraint"]["by_zone"]["destinations"] = dest
    compute_matrix(calc1)
    export_matrix(d1)
    os.rename(os.path.join(inputFolder, d1 + ".csv"),os.path.join(inputFolder, dest + "_peak.csv"))
    
    calc2 = deepcopy(calcSpec)
    calc2["expression"] = opAutoTime
    calc2["result"] = d1
    calc2["aggregation"]["origins"] = "+"
    calc2["constraint"]["by_zone"]["destinations"] = dest
    compute_matrix(calc2)
    export_matrix(d1)
    os.rename(os.path.join(inputFolder, d1 + ".csv"),os.path.join(inputFolder, dest + "_offpeak.csv"))

In [11]:
timeMatrices = ["mf81","mf82","mf83","mf84"] # 311 files have to be hardcoded - could fix this by reading the files - look at sixth line
for root, dirs, files in os.walk(os.path.join(inputFolder,"TransitTime")):
    for matrix_file in files:
        if matrix_file.endswith(".mdf"):
            matrix_number = eb.available_matrix_identifier('FULL')
            timeMatrices.append(matrix_number)
            import_matrices(4, matrix_number[2:] ,os.path.join(root, matrix_file),mm.scenario, matrix_file[:-4] )
            change_matrix(matrix = matrix_number,
                          matrix_name = matrix_file[:-4],
                          matrix_description = matrix_file[:-4])    
        elif matrix_file.endswith(".311"):
            matrix311(transaction_file = os.path.join(root, matrix_file),
                      throw_on_error = True,
                      scenario = mm.scenario)

In [12]:
minExpression = ""
for mat in timeMatrices:
    if timeMatrices.index(mat) == 0:
        minExpression = minExpression + mat
    else:
        minExpression = minExpression + " .min. " + mat
    calc = deepcopy(calcSpec)
    calc["expression"] = "9999"
    calc["result"] = mat
    calc["constraint"]["by_value"] = {"interval_min": 0,
                                   "interval_max": 0,
                                   "condition": "INCLUDE",
                                   "od_values": mat}
    compute_matrix(calc)

timeMat = create_matrix(matrix_id ="mf",
              matrix_name = "very_new_time_transit_peak" + year,
              matrix_description = "Nov Peak Transit Time" + year)

tranTime = timeMat.id

minCalc = deepcopy(calcSpec)
minCalc["expression"] = minExpression
minCalc["result"] = tranTime

compute_matrix(minCalc)

for mat in timeMatrices:
    delete_matrix(mat,eb)
    
filterCalc = deepcopy(calcSpec)
filterCalc["expression"] = "0"
filterCalc["result"] = tranTime
filterCalc["constraint"]["by_value"] = {"interval_min": 0,
                                   "interval_max": 180,
                                   "condition": "EXCLUDE",
                                   "od_values": tranTime}
compute_matrix(filterCalc)

autofilterCalc = deepcopy(calcSpec)
autofilterCalc["expression"] = "0"
autofilterCalc["result"] = autoTime
autofilterCalc["constraint"]["by_value"] = {"interval_min": 0,
                                   "interval_max": 0,
                                   "condition": "INCLUDE",
                                   "od_values": tranTime}
compute_matrix(autofilterCalc)

{'average': 0.0,
 'expression_type': 'FULL',
 'maximum': 0.0,
 'maximum_at': {'destination': 1001, 'origin': 1001},
 'minimum': 0.0,
 'minimum_at': {'destination': 1001, 'origin': 1001},
 'num_evaluations': 5889689,
 'result_type': 'FULL',
 'sum': 0.0}

In [17]:
demand = "mf27" #2016
#demand = "mf22" #2041C
#demand = "mf17"#2041CP

tranTime = "mf36" #2016
#tranTime = "mf26" #2041C
#tranTime = "mf21" #2041CP

autoTime = "mf28" #2016
#autoTime = "mf18" #2041C
#autoTime = "mf14" #2041CP


In [18]:
d1 = "md31"
d2 = "md32"
d3 = "md33"
d4 = "md34"
d5 = "md35"
#d6 = finalDest

"""
create_matrix(matrix_id = d1,
              matrix_name = "newtimedem_2041CP",
              matrix_description = "Time Demand Dest Matrix Committed Plus Transit")

create_matrix(matrix_id = d2,
              matrix_name = "autotransdem_2041CP",
              matrix_description = "Demand Dest Matrix Committed Plus Transit and Auto")

create_matrix(matrix_id = d3,
              matrix_name = "newtimefinal_2041CP",
              matrix_description = "New Final Weighted Time Transit Dest Matrix Committed Plus")

create_matrix(matrix_id = d4,
              matrix_name = "autonewtimedem_2041CP",
              matrix_description = "Time Demand Dest Matrix Committed Plus Auto")

create_matrix(matrix_id = d5,
              matrix_name = "autonewtimefinal_2041CP",
              matrix_description = "New Final Weighted Time Auto Dest Matrix Committed Plus")

create_matrix(matrix_id = d6,
              matrix_name = "newAutoTransitRatioCP",
              matrix_description = "New Final Auto Transit Ratio Committed Plus")
"""
calc1 = deepcopy(calcSpec)
calc1["expression"] = demand + "*" + tranTime
calc1["result"] = d1
calc1["aggregation"]["origins"] = "+"

compute_matrix(calc1)

calc2 = deepcopy(calcSpec)
calc2["expression"] = demand 
calc2["result"] = d2
calc2["aggregation"]["origins"] = "+"
calc2["constraint"]["by_value"] = {"interval_min": 0,
                                   "interval_max": 0,
                                   "condition": "EXCLUDE",
                                   "od_values": tranTime}

compute_matrix(calc2)

calc3 = deepcopy(calcSpec)
calc3["expression"] = d1 + "/" + d2
calc3["result"] = d3
calc3["constraint"]["by_value"] = {"interval_min": 0,
                                   "interval_max": 0,
                                   "condition": "EXCLUDE",
                                   "od_values": d2}
compute_matrix(calc3)

calc4 = deepcopy(calcSpec)
calc4["expression"] = demand + "*" + autoTime
calc4["result"] = d4
calc4["aggregation"]["origins"] = "+"
compute_matrix(calc4)

calc5 = deepcopy(calcSpec)
calc5["expression"] = d4 + "/" + d2
calc5["result"] = d5
calc5["constraint"]["by_value"] = {"interval_min": 0,
                                   "interval_max": 0,
                                   "condition": "EXCLUDE",
                                   "od_values": d2}
compute_matrix(calc5)

"""
calc6 = deepcopy(calcSpec)
calc6["expression"] = d5 + "/" + d3
calc6["result"] = d6
calc6["constraint"]["by_value"] = {"interval_min": 0,
                                   "interval_max": 0.1,
                                   "condition": "EXCLUDE",
                                   "od_values": d3}

create_matrix(matrix_id = d6,
              matrix_name = "newAutoTransitRatio" + year,
              matrix_description = "New Final Auto Transit Ratio Committed Plus " + year)
compute_matrix(calc6)
"""

'\ncalc6 = deepcopy(calcSpec)\ncalc6["expression"] = d5 + "/" + d3\ncalc6["result"] = d6\ncalc6["constraint"]["by_value"] = {"interval_min": 0,\n                                   "interval_max": 0.1,\n                                   "condition": "EXCLUDE",\n                                   "od_values": d3}\n\ncreate_matrix(matrix_id = d6,\n              matrix_name = "newAutoTransitRatio" + year,\n              matrix_description = "New Final Auto Transit Ratio Committed Plus " + year)\ncompute_matrix(calc6)\n'